## Assignment 2

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
 
data = pd.read_csv('../household_power_consumption.txt', sep=";",parse_dates=[['Date', 'Time']], header=0, dtype={"Global_active_power": float, 
                                                                                'Global_reactive_power': float,
                                                                               'Voltage': float, 'Global_intensity': float,
                                                                               'Sub_metering_1': float, 'Sub_metering_2': float, 'Sub_metering_3':float})


data.dropna(inplace=True)
data.set_index('Date_Time')
data[:20]

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
def plotGraphs(col):
    plt.figure()
    plt.plot(data['Date_Time'], data[col])
    startDate = datetime.date(2007, 1, 1)
    endDate = datetime.date(2007, 12, 31)
    ax.set_xlim(startDate, endDate)
    plt.title(col)
    plt.gcf().autofmt_xdate() # Rotation
    plt.show()

In [ ]:
plotGraphs('Global_active_power')
plotGraphs('Global_reactive_power')
plotGraphs('Voltage')


In [ ]:
def normalize(col):
    mean = np.mean(data[col])
    std = np.std(data[col])
    mean,std
    data[col] = (data[col]-mean)/std

In [ ]:
normalize('Global_active_power')
normalize('Global_reactive_power')
normalize('Voltage')
normalize('Global_intensity')
normalize('Sub_metering_1')
normalize('Sub_metering_2')
normalize('Sub_metering_3')
data[:20]

In [ ]:
def Preprocess(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
processedData = Preprocess(data, 1, 1)

In [ ]:
processedData[:20]

## Question 2:

In [ ]:
# 913 is 2 and half years to train and the rest for testing 
# split into train and test sets
values = processedData.values
n_train_hours = 913 * 24 * 60
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=1, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()